In [8]:
from dlhub_sdk.models.servables.keras import KerasModel
from dlhub_sdk import DLHubClient
import pickle as pkl
import json

# Train a Simple Keras Example on MNIST

In [4]:
## Copied from Keras examples
"""Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.

Taken from: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

Modified to:
    - Use only 1024 examples for faster training
    - Save model to hd5 at the end
"""

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12
train_size = 512

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train[:train_size], y_train[:train_size],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model.save("model.hd5")


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 512 samples, validate on 10000 samples
Epoch 1/12
512/512 [==============================] - 2s 5ms/step - loss: 2.1108 - acc: 0.2812 - val_loss: 1.6300 - val_acc: 0.6573
Epoch 2/12
512/512 [==============================] - 2s 4ms/step - loss: 1.8010 - acc: 0.4590 - val_loss: 1.2421 - val_acc: 0.6713
Epoch 3/12
512/512 [==============================] - 2s 4ms/step - loss: 1.1886 - acc: 0.6309 - val_loss: 1.2574 - val_acc: 0.5668
Epoch 4/12
512/512 [==============================] - 2s 4ms/step - loss: 0.9311 - acc: 0.7188 - val_loss: 0.8100 - val_acc: 0.7381
Epoch 5/12
512/512 [==============================] - 2s 4ms/step - loss: 0.6493 - acc: 0.7910 - val_loss: 0.9804 - val_acc: 0.6577
Epoch 6/12
512/512 [==============================] - 2s 4ms/step - loss: 0.6824 - acc: 0.7852 - val_loss: 0.7639 - val_acc: 0.7782
Epoch 7/12
512/512 [==============================] - 2s 4ms/step - loss: 0.5329 - acc:

# Describe the Model

In [5]:
# Describe the keras model
model_info = KerasModel.create_model('model.hd5', list(map(str, range(10))))

#    Describe the model
model_info.set_title("MNIST Digit Classifier")
model_info.set_name("mnist_tiny_example")
model_info.set_domains(["general", "digit recognition"])

#    Describe the outputs in more detail
model_info['servable']['methods']['run']['output']['description'] = 'Probabilities of being 0-9'
model_info['servable']['methods']['run']['input']['description'] = 'Image of a digit'

# Print out the result
print('\n--> Model Information <--')
print(json.dumps(model_info.to_dict(), indent=2))

# Save the model information to pickle
with open('model_info.pkl', 'wb') as fp:
    pkl.dump(model_info, fp)


--> Model Information <--
{
  "datacite": {
    "creators": [],
    "titles": [
      {
        "title": "MNIST Digit Classifier"
      }
    ],
    "publisher": "DLHub",
    "publicationYear": "2019",
    "identifier": {
      "identifier": "10.YET/UNASSIGNED",
      "identifierType": "DOI"
    },
    "descriptions": [],
    "fundingReferences": [],
    "relatedIdentifiers": [],
    "alternateIdentifiers": [],
    "rightsList": [],
    "resourceType": {
      "resourceTypeGeneral": "InteractiveResource"
    }
  },
  "dlhub": {
    "version": "0.8.2",
    "domains": [
      "general",
      "digit recognition"
    ],
    "visible_to": [
      "public"
    ],
    "name": "mnist_tiny_example",
    "files": {
      "model": "model.hd5"
    },
    "type": "servable",
    "dependencies": {
      "python": {
        "keras": "2.2.4",
        "h5py": "2.8.0",
        "tensorflow": "1.14.0"
      }
    }
  },
  "servable": {
    "methods": {
      "run": {
        "input": {
          "type":

# Run this to Publish the Model

In [ ]:
dl = DLHubClient()

In [9]:
res = dl.publish_servable(model_info)
res

In [16]:
dl.get_task_status(res)

{'status': None, 'result': ''}